In [11]:
import requests  # Untuk mengambil data dari API
import pandas as pd  # Untuk manipulasi data
import seaborn as sns  # Untuk visualisasi data
from sklearn.linear_model import LinearRegression  # Untuk model regresi linier
import numpy as np  # Untuk operasi numerik
import networkx as nx  # Untuk analisis jejaring sosial
import folium  # Untuk visualisasi data geografis
from IPython.display import display  # Untuk menampilkan output di Jupyter Notebook
import ipywidgets as widgets  # Untuk membuat widget interaktif
import matplotlib.pyplot as plt  # Untuk visualisasi data

# URL API
url = "https://opendata.bekasikab.go.id/api/bigdata/dp/hrg_brng_pkk_brdsrkn_kmdts_trwln_k_1_st_d_kbptn_bks_thn_2023"

# Mengambil data dari API
try:
    response = requests.get(url, verify=False)
    response.raise_for_status()  # Menangani error HTTP
    data = response.json()
except requests.exceptions.RequestException as e:
    print(f"Failed to fetch data from API: {e}")
    data = None

if data:
    # Konversi data ke DataFrame dengan timestamp real-time
    df = pd.DataFrame(data['data'])
    current_time = pd.Timestamp.now()
    
    # Tampilkan beberapa baris pertama dengan timestamp
    print(f"Data retrieved at: {current_time}")
    print(df.head())

    # Pembersihan Data
    # Mengubah kolom tanggal menjadi tipe datetime dengan timestamp sekarang
    df['tanggal'] = pd.to_datetime(df['tanggal'])
    df['retrieved_at'] = current_time

    # Mengubah kolom harga_komoditas menjadi tipe numerik
    df['harga_komoditas'] = pd.to_numeric(df['harga_komoditas'], errors='coerce')

    # Menghapus baris dengan nilai NaN
    df = df.dropna()

    # Analisis Data Real-time
    # Rata-rata harga per komoditas dengan timestamp
    avg_price = df.groupby('nama_komoditas').agg({
        'harga_komoditas': 'mean',
        'retrieved_at': 'max'
    }).reset_index()

    # Plot rata-rata harga per komoditas
    plt.figure(figsize=(10, 6))
    sns.barplot(x='nama_komoditas', y='harga_komoditas', data=avg_price)
    plt.title(f'Rata-rata Harga per Komoditas (as of {current_time.strftime("%Y-%m-%d %H:%M:%S")})')
    plt.xlabel('Komoditas')
    plt.ylabel('Harga')
    plt.xticks(rotation=45)
    plt.show()

    # B. Analisis Jejaring Sosial Real-time
    # Membuat graf kosong
    G = nx.Graph()

    # Menambahkan node (wilayah) dan edge (hubungan harga antar wilayah) dengan timestamp
    hubungan_harga = [
        ('KABUPATEN BEKASI', 'KOTA BEKASI', 0.9),
        ('KABUPATEN BEKASI', 'KOTA BOGOR', 0.8),
        ('KOTA BEKASI', 'KOTA BOGOR', 0.85),
    ]

    # Menambahkan node dan edge ke graf
    for wilayah1, wilayah2, hubungan in hubungan_harga:
        G.add_edge(wilayah1, wilayah2, weight=hubungan)

    # Mengidentifikasi aktor penting menggunakan centrality measures secara real-time
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)

    # Membuat DataFrame untuk analisis real-time
    real_time_analytics = pd.DataFrame({
        'Wilayah': list(degree_centrality.keys()),
        'Degree Centrality': list(degree_centrality.values()),
        'Betweenness Centrality': list(betweenness_centrality.values()),
        'Closeness Centrality': list(closeness_centrality.values()),
        'Timestamp': current_time,
    })

    # Menghitung total influence score
    real_time_analytics['Total Influence Score'] = (
        real_time_analytics['Degree Centrality'] + 
        real_time_analytics['Betweenness Centrality'] + 
        real_time_analytics['Closeness Centrality']
    ) / 3

    # Menampilkan hasil analisis real-time
    print("\nReal-time Network Analysis Results:")
    print(real_time_analytics)

    # Visualisasi graf dengan timestamp
    plt.figure(figsize=(10, 6))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_size=700, node_color='skyblue', font_size=10, font_weight='bold')
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
    plt.title(f'Grafik Hubungan Harga Antar Wilayah (as of {current_time.strftime("%Y-%m-%d %H:%M:%S")})')
    plt.show()

    # C. Regresi Linier Real-time
    # Menggunakan data historis untuk membangun model regresi
    df['dayofyear'] = df['tanggal'].dt.dayofyear
    X = df[['dayofyear']]
    y = df['harga_komoditas']
    
    # Melatih model dengan timestamp
    model = LinearRegression()
    model.fit(X, y)
    
    # Prediksi harga di masa depan
    future_dates = pd.date_range(start=pd.Timestamp.now().strftime('%Y-%m-%d'), periods=365)
    future_days = future_dates.dayofyear
    
    # Pastikan input prediksi memiliki nama fitur yang sama
    future_days_df = pd.DataFrame(future_days, columns=['dayofyear'])
    predictions = model.predict(future_days_df)

    # Buat DataFrame untuk prediksi
    # Get current date as starting point for predictions
    real_time_date = '2023-01-08' # Just for example, you can use actual current date
    real_time_day_of_year = pd.to_datetime(real_time_date).dayofyear
    
    # Make real-time prediction for current date
    real_time_prediction = model.predict(pd.DataFrame([[real_time_day_of_year]], columns=['dayofyear']))
    
    # Create DataFrame for future predictions starting from current date
    future_df = pd.DataFrame({
        'Tanggal': future_dates,
        'Harga_Prediksi': predictions
    })

    # Plot prediksi harga
    plt.figure(figsize=(10, 6))
    plt.plot(future_df['Tanggal'], future_df['Harga_Prediksi'], label='Prediksi Harga')
    plt.axvline(x=pd.to_datetime(real_time_date), color='r', linestyle='--', label='Tanggal')
    plt.title('Prediksi Harga Pangan untuk 2025')
    plt.xlabel('Tanggal')
    plt.ylabel('Harga')
    plt.legend()
    plt.show()

    # D. Visualisasi Big Data dengan Lokasi Real-time
    # Membuat peta dasar dengan lokasi tengah Kabupaten Bekasi
    m = folium.Map(location=[-6.241586, 107.000000], zoom_start=10)

    # Menambahkan marker untuk setiap lokasi dengan data real-time
    for idx, row in df.iterrows():
        # Menggunakan data harga terkini dengan timestamp
        popup_text = f"""
        Komoditas: {row['nama_komoditas']}
        Harga: Rp {row['harga_komoditas']:,}
        Tanggal: {row['tanggal'].strftime('%Y-%m-%d')}
        Update: {current_time.strftime('%Y-%m-%d %H:%M:%S')}
        """
        
        # Menambahkan marker dengan informasi real-time
        folium.Marker(
            location=[-6.241586, 107.000000],  # Koordinat Kabupaten Bekasi
            popup=folium.Popup(popup_text, max_width=300),
            tooltip=f"{row['nama_komoditas']} - Rp {row['harga_komoditas']:,}",
        ).add_to(m)

    # Menampilkan peta dengan timestamp
    print(f"Map updated at: {current_time}")
    display(m)
else:
    print("No data available to process.")

Failed to fetch data from API: HTTPSConnectionPool(host='opendata.bekasikab.go.id', port=443): Max retries exceeded with url: /api/bigdata/dp/hrg_brng_pkk_brdsrkn_kmdts_trwln_k_1_st_d_kbptn_bks_thn_2023 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000020780720980>: Failed to resolve 'opendata.bekasikab.go.id' ([Errno 11002] getaddrinfo failed)"))
No data available to process.
